# Обработка данных по KPI УЗ

Включение и отключение рассылки для тестов:

In [13]:
send_mail = True
mail_to_all = True
target_week = 42

# Ссылка на файл в Nextcloud с общим доступом
url = "https://cloud.melonfashion.ru/s/yqtGwP6LJdYwb76/download/KPI%20%D0%A3%D1%82%D0%BA%D0%B8%D0%BD%D0%B0%20%D0%97%D0%B0%D0%B2%D0%BE%D0%B4%D1%8C%20v%202.xlsx"
user_url = "https://cloud.melonfashion.ru/f/75879760"

Импорты

In [14]:
import pandas as pd
import warnings
import requests
import numpy as np
import smtplib
import datetime
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.utils import get_column_letter
from openpyxl.styles import Alignment
from openpyxl import load_workbook
from openpyxl.styles import Alignment, PatternFill, Font

warnings.filterwarnings("ignore")

### Подключение к данным

In [15]:
data_update_time = datetime.datetime.now().strftime("%Y.%m.%d_%H.%M.%S")

# Скачивание файла
response = requests.get(url)
file_name = "downloaded_file.xlsx"

# Сохранение файла на диск
with open(file_name, "wb") as file:
    file.write(response.content)

#### df_errors

In [16]:
# Формирование датафрейма с описаниями отклонений
df_errors = pd.read_excel(file_name, 1)[
    [
        "ID KPI",
        "Бренд",
        "Неделя",
        "Значение KPI",
        "Категория",
        "Краткое описание причины",
    ]
]
df_errors["ID KPI"] = df_errors["ID KPI"].replace(np.NaN, "Other")
df_errors["Категория"] = df_errors["Категория"].replace(np.NaN, "Отклонение")
df_errors["Бренд"] = df_errors["Бренд"].replace(np.NaN, "МФГ")
df_errors.dropna(inplace=True)
df_errors["Неделя"] = df_errors["Неделя"].astype(int)

# Устранение дубликатов по KPI и неделе в df_errors
df_errors["Краткое описание причины"] = (
    df_errors["Бренд"] + ": " + df_errors["Краткое описание причины"] + ";"
)
df_errors = df_errors.groupby(["ID KPI", "Неделя", "Категория"], as_index=False).agg(
    {"Краткое описание причины": " \n".join}
)
df_errors.columns = ["id", "week", "category", "description"]

df_errors.to_csv("tableau/df_errors.csv", sep="\t", index_label="index")
df_errors.to_csv(
    f"logs/tableau/df_errors_{data_update_time}.csv", sep="\t", index_label="index"
)

#### df_base

In [17]:
# Формирование датафрейма с значениями показателей
df_base = pd.read_excel(file_name, 2)

df_base.to_csv("tableau/df_base.csv", sep="\t", index_label="index")
df_base.to_csv(
    f"logs/tableau/df_base_{data_update_time}.csv", sep="\t", index_label="index"
)

#### df_values

In [18]:
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False


# Формирование датафрейма с значениями показателей
df_values = pd.read_excel(file_name)
df_values = df_values[df_values["Бренд"] == "Итого"].replace("-", np.NaN)

weeks = ["id"] + [x for x in range(1, 52)]

# Преобразуем в вертикальную таблицу
df_pivot = df_values.loc[:, weeks].melt(id_vars="id")
df_pivot.columns = ["id", "week", "value"]
# Преобразуем тип данных
df_pivot["week"] = pd.to_numeric(df_pivot["week"], errors="raise")

# Применим функцию ко всему столбцу и создадим маску
mask = df_pivot["value"].apply(is_number)

# Отфильтруем DataFrame
df_values = df_pivot[mask]

# Преобразуем значения в числовой тип
df_values["value"] = df_values["value"].astype(float)

df_values.to_csv("tableau/df_values.csv", sep="\t", index_label="index")
df_values.to_csv(
    f"logs/tableau/df_values_{data_update_time}.csv", sep="\t", index_label="index"
)

## Подготовка рассылки

### kpi_base

In [19]:
kpi_base = df_base[df_base["relevance"] == 1][
    ["id", "title", "department", "units", "responsible", "target", "target_type"]
]

kpi_base.head()

,id,title,department,units,responsible,target,target_type
0,KPI01,Своевременность выгрузки.,Склад товара,% (ТС = Транспортное средство),Оператор,0.98,higher
1,KPI02,Своевременность выгрузки.,Склад оборудования,% (ТС = Транспортное средство),Оператор,0.98,higher
2,KPI03,Своевременность погрузки.,Склад товара,% (ТС = Транспортное средство),Оператор,0.98,higher
3,KPI04,Количество полученных возвратов с розницы.,Склад товара,шт. (штуки),Оператор,5000.00,lower
4,KPI05,Количество принятых возвратов с розницы.,Склад товара,шт. (штуки),Оператор,5000.00,higher


### kpi_errors

In [20]:
kpi_errors = df_errors.copy()
kpi_errors.columns = ["id", "week", "category", "description"]

kpi_errors.head()

,id,week,category,description
0,KPI01,19,Инцидент,МФГ: Не своевременно разгрузка товара со склад...
1,KPI01,24,Отклонение,МФГ: Не своевременно разгрузка товара со склад...
2,KPI01,25,Отклонение,"МФГ: В связи с большими расхождениями, пересчи..."
3,KPI01,34,Отклонение,"МФГ: Сотрудник допустил ошибку, не завершив за..."
4,KPI01,35,Отклонение,"МФГ: По заданиям 000012913, 000012997 несвоевр..."


### df_tableau

In [21]:
# Подготовим результирующую таблицу
df_tableau = pd.merge(left=df_values, right=kpi_base, on="id", how="left")
df_tableau = pd.merge(left=df_tableau, right=kpi_errors, on=["id", "week"], how="left")
df_tableau.fillna(0)

df_tableau.head()

,id,week,value,title,department,units,responsible,target,target_type,category,description
0,KPI01,1,1.0,Своевременность выгрузки.,Склад товара,% (ТС = Транспортное средство),Оператор,0.98,higher,NaN,NaN
1,KPI02,1,1.0,Своевременность выгрузки.,Склад оборудования,% (ТС = Транспортное средство),Оператор,0.98,higher,NaN,NaN
2,KPI03,1,NaN,Своевременность погрузки.,Склад товара,% (ТС = Транспортное средство),Оператор,0.98,higher,NaN,NaN
3,KPI04,1,53254.0,Количество полученных возвратов с розницы.,Склад товара,шт. (штуки),Оператор,5000.00,lower,NaN,NaN
4,KPI05,1,25429.0,Количество принятых возвратов с розницы.,Склад товара,шт. (штуки),Оператор,5000.00,higher,NaN,NaN


### Функции

In [22]:
def is_good_value(row) -> bool:
    if (
        row["target_type"] != "higher"
        and row["target_type"] != "lower"
        and row["value"]
    ):
        return True
    elif row["target_type"] == "higher" and row["value"] >= row["target"]:
        return True
    elif row["target_type"] == "lower" and row["value"] <= row["target"]:
        return True
    else:
        return False


def create_weekly_report(week):
    df_mail = (
        df_tableau[df_tableau["week"] == week]
        .sort_values(by="id")
        .reset_index(drop=True)
    )
    df_mail["is_good"] = df_mail.apply(lambda row: is_good_value(row), axis=1)

    df_mail = df_mail[df_mail.is_good == False]
    df_mail.set_index("responsible", inplace=True)
    df_mail.columns = [
        "KPI",
        "Неделя",
        "Значение",
        "Описание",
        "Склад",
        "Измерение",
        "Цель",
        "Тип цели",
        "Категория",
        "Причина отклонения",
        "xx",
    ]
    df_mail = df_mail[
        [
            "KPI",
            "Описание",
            "Измерение",
            "Склад",
            "Неделя",
            "Цель",
            "Значение",
            "Причина отклонения",
        ]
    ]
    return df_mail

### Форматирование файла перед отправкой

In [23]:
def format_book(name: str):
    # Load the workbook and select the active worksheet
    wb = load_workbook(filename=name)
    ws = wb.active

    # Define formatting styles
    center_alignment = Alignment(horizontal="center", vertical="center")
    header_fill = PatternFill(
        start_color="ADD8E6", end_color="ADD8E6", fill_type="solid"
    )
    header_font = Font(bold=True)
    yellow_fill = PatternFill(
        start_color="FFFF00", end_color="FFFF00", fill_type="solid"
    )

    # Apply yellow fill for missing data and round numbers to 2 decimal places for relevant columns
    for row in ws.iter_rows(
        min_row=2, max_row=ws.max_row, min_col=1, max_col=ws.max_column
    ):
        for cell in row:
            if (
                cell.column == 8 or cell.column == 9
            ):  # "Значение" and "Причина отклонения"
                if cell.value is None or cell.value == "":
                    cell.fill = yellow_fill
            if cell.column == 7 and isinstance(cell.value, (float, int)):
                cell.value = round(cell.value, 2)  # Round to 2 decimal places
            if cell.column == 8 and isinstance(cell.value, (float, int)):
                cell.value = round(cell.value, 2)  # Round to 2 decimal places

            if cell.column in [
                1,
                2,
                6,
                7,
                8,
            ]:  # Columns "№", "Неделя", "Значение", "Ед.изм-я", "Порог"
                cell.alignment = center_alignment
            if cell.column in [
                3,
                4,
                5,
                9,
            ]:  # Apply text wrapping to "Причина отклонения"
                cell.alignment = Alignment(
                    horizontal="left", vertical="center", wrap_text=True
                )

    # Set fixed column widths
    column_widths = [18, 12, 50, 25, 25, 15, 15, 15, 80]  # Adjusted for readability
    for i, column_width in enumerate(column_widths, start=1):
        ws.column_dimensions[get_column_letter(i)].width = column_width

    # Apply header row styling
    for cell in ws[1]:
        cell.fill = header_fill
        cell.font = header_font
        cell.alignment = center_alignment

    # Remove gridlines
    ws.sheet_view.showGridLines = True

    # Save the formatted file
    output_path = name
    wb.save(output_path)

## Рассылка запросов по фиксации отклонений

In [24]:
creating_report_time = datetime.datetime.now().strftime("%Y.%m.%d_%H.%M.%S")
filename = f"logs/mail/week_{target_week}_on_{creating_report_time}.xlsx"
file_path = f"logs/mail/week_{target_week}_on_{creating_report_time}.xlsx"

# Подготовка отчета
create_weekly_report(target_week).to_excel(filename)
format_book(filename)

# Настройки отправки почты
sender_email = "orinel1995@gmail.com"
receiver_email = "kondratyevai@melonfashion.com"
receiver_emails = [
    "kirikyav@melonfashion.com",
    "uchetUZ@melonfashion.com",
    "orinel1995@gmail.com",
    "polyakovada@melonfashion.com",
    "operatorUZ@melonfashion.com",
    "claims.sklad.uzavod@melonfashion.com",
]
subject = f"Отчет по не заполненным KPI."
body = f"Добрый день. \n\nВо вложении результат заполнения данных за {target_week} неделю. Прошу заполнить пропуски в таблицы MelonCloud: \n{user_url} \n\n С уважением, Александр Кондратьев."

if send_mail:
    # Создание объекта MIMEMultipart
    msg = MIMEMultipart()
    msg["From"] = sender_email
    if mail_to_all:
        msg["To"] = receiver_email + ", " + ", ".join(receiver_emails)
    else:
        msg["To"] = receiver_email

    msg["Subject"] = subject

    # Добавление текста письма
    msg.attach(MIMEText(body, "plain"))

    # Чтение файла и добавление его в письмо
    with open(file_path, "rb") as file:
        part = MIMEApplication(file.read(), Name=filename)
        part["Content-Disposition"] = f'attachment; filename="{filename}"'
        msg.attach(part)

    # Отправка письма через SMTP-сервер
    with smtplib.SMTP("smtp.gmail.com", 587) as server:
        server.starttls()
        server.login(sender_email, "cckd uvwc ftfh prbv")
        server.send_message(msg)

    print("Письмо отправлено успешно.")


Письмо отправлено успешно.
